# Isogenies

**Note: due to the context of this being explained during Sage days, I assume you are all running the most up to date version of SageMath at the time of writing (version 10.2)**

This notebook assumes you have already gone through, or are familiar with using elliptic curves in Sage. 

## Isomorphisms

As we have done half the work here already, let us continue looking at elliptic curves and isogenies by first considering isomorphisms between curves and points.

Unless otherwise stated, we will work with the following finite field and curve:

In [2]:
a, b = 13, 7
p = 2^a * 3^b - 1
F.<i> = GF(p**2, modulus=[1,0,1])

E = EllipticCurve(F, [0, 6, 0, 1, 0])
print(f"{E = }")
P, Q = E.gens()
print(f"{P = }")
print(f"{Q = }")

E = Elliptic Curve defined by y^2 = x^3 + 6*x^2 + x over Finite Field in i of size 17915903^2
P = (12772366*i + 5961474 : 5144390*i + 13478759 : 1)
Q = (8260143*i + 518463 : 2768119*i + 4146976 : 1)


## Mapping to Short Weierstrass Form

Let's first look at how we can map from the Montgomery model to a short Weierstrass curve.

In [3]:
# Given the isomorphic curve in the Short Weierstrass model
# we can compute the isomorphism with the isomorphism_to method
E_short = E.short_weierstrass_model()
iso_to_E_short = E.isomorphism_to(E_short)
print(iso_to_E_short)

Elliptic-curve morphism:
  From: Elliptic Curve defined by y^2 = x^3 + 6*x^2 + x over Finite Field in i of size 17915903^2
  To:   Elliptic Curve defined by y^2 = x^3 + 17901647*x + 653184 over Finite Field in i of size 17915903^2
  Via:  (u,r,s,t) = (14929919, 17915901, 0, 0)


In [5]:
# We can also map points on one curve to another like so:
P_short = iso_to_E_short(P)
Q_short = iso_to_E_short(Q)
print(f"{P_short = }")
print(f"{Q_short = }")
print(f"{P_short.curve() = }")

P_short = (11907601*i + 17538203 : 17513649*i + 8880245 : 1)
Q_short = (10710700*i + 748837 : 11226998*i + 48334 : 1)
P_short.curve() = Elliptic Curve defined by y^2 = x^3 + 17901647*x + 653184 over Finite Field in i of size 17915903^2


## Mapping to the Montgomery Model

The above works for any two isomorphic curves. As long as `E.is_isomorphic(E_prime)` returns `True`, then `isomorphism_to()` will return the morphism between curves with a `domain` equal to `E` and `codomain` equal to `E_prime.`

For mapping to the Montgomery model, there's actually an even easier method thanks to a relatively recent function. Let's see how it works. Calling the following function returns the correct codomain:

In [7]:
E_mont = E_short.montgomery_model()
print(f"{E_mont = }")
print(f"{E_mont.is_isomorphic(E) = }")

E_mont = Elliptic Curve defined by y^2 = x^3 + 14772668*x^2 + x over Finite Field in i of size 17915903^2
E_mont.is_isomorphic(E) = True


However, if we want the morphism between these two curves instead, we can use the following optional argument

In [10]:
(E_mont, iso_to_mont) = E_short.montgomery_model(morphism=True)
iso_to_mont

Elliptic-curve morphism:
  From: Elliptic Curve defined by y^2 = x^3 + 17901647*x + 653184 over Finite Field in i of size 17915903^2
  To:   Elliptic Curve defined by y^2 = x^3 + 14772668*x^2 + x over Finite Field in i of size 17915903^2
  Via:  (u,r,s,t) = (12422522, 16275630, 0, 0)

Now, instead of returning the codomain, we also get the isomorphism between curves. Just like the above, we can map points through this morphism

In [12]:
print(f"{iso_to_mont(P) = }")
print(f"{iso_to_mont(Q) = }")

iso_to_mont(P) = (323911*i + 10826241 : 7815892*i + 2464233 : 1)
iso_to_mont(Q) = (6301774*i + 2588315 : 3816725*i + 219936 : 1)


## Isogenies from Points

## Isogenies from Kernel Polynomials

## Isogenies Between Curves

## Smooth Degree Isogenies

## Large Degree Isogenies